# 📂 Cross Validation（交叉验证）

---

## 📘 介绍

在机器学习中，我们的最终目标是构建一个**泛化能力强**的模型，即在未见数据上也能保持良好性能。  
但如果我们只是简单将数据划分为训练集 / 测试集一次评估，就容易受到：

- 数据划分的**随机性影响**（比如刚好测试集中是噪声点）
- 模型对某些数据划分**过拟合**或**欠拟合**
- 评估结果**波动大，不可靠**

为了解决这个问题，引入了 **交叉验证（Cross Validation）**：  
> 将数据多次切分、循环训练与验证，最终对性能进行平均评估，从而更稳定、更可靠地估计模型的泛化能力。

---

## 🧠 和 Holdout 的区别

| 方法         | 特点                                               |
|--------------|----------------------------------------------------|
| Holdout      | 数据一次划分为训练集和验证集，快速但受随机影响大      |
| Cross Validation | 多次划分、多轮验证，**更稳定，更能代表模型真实性能** |

---

## ✅ 交叉验证常用于：

- 模型选择与比较（哪种模型更稳定？）
- 参数调优（GridSearchCV / RandomizedSearchCV）
- 特征工程阶段评估新特征是否有效
- 控制过拟合，判断模型是否泛化过强或过弱

---

## 🧪 常见交叉验证方法概览

| 方法名                 | 适用场景               |
|------------------------|------------------------|
| K-Fold                 | 通用、高频、回归 / 分类任务       |
| Stratified K-Fold      | 样本不均衡分类任务             |
| Leave-One-Out (LOOCV)  | 样本少、无偏估计               |
| Group K-Fold           | 分组样本，不能泄露组内信息        |
| Time Series Split      | 时间序列预测任务，保留时间顺序     |



## 🔹 Train/Test Split（Holdout）

---

### 📘 方法介绍

Train/Test Split（也称 Holdout 法）是最基础的验证方法：

> 将原始数据集一次性划分为训练集（train set）与测试集（test set），然后在训练集上训练模型，在测试集上评估模型性能。

常见划分比例：
- 回归任务：80% 训练 / 20% 测试
- 小样本场景：70/30、甚至 60/40 也常见

---

### 📐 示例流程

```text
原始数据集（1000 个样本）
→ 训练集（800）用于拟合模型
→ 测试集（200）用于评估泛化能力


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut

In [15]:
# 加载数据
X, y = fetch_california_housing(return_X_y=True)

# 打印原始数据形状
print("原始数据集大小:", X.shape)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 打印划分后形状
print("训练集大小:", X_train.shape)
print("测试集大小:", X_test.shape)


原始数据集大小: (20640, 8)
训练集大小: (16512, 8)
测试集大小: (4128, 8)


## 🔁 K-Fold Cross Validation（K折交叉验证）

---

### 📘 介绍

K-Fold 是最常用的一种交叉验证方法。

> 将数据集划分为 K 个等份（folds），每次选取一个 fold 作为验证集，其余 K-1 个 fold 作为训练集。重复 K 次，最终将 K 次评估结果求平均，作为模型的最终性能表现。

---

### ⚙️ 原理

- 将整个数据集平均分为 K 份
- 执行 K 次训练-验证：
  - 每次留出一份做验证集
  - 其余 K-1 份用于训练
- 最后对 K 次模型评估结果取平均，作为整体性能评估

---

### 🧠 特性

- 每个样本都有一次成为验证集的机会
- 更可靠、更稳定地估计模型泛化能力
- 充分利用数据，提高训练效果

---

### ✅ 优点

- 评估结果更稳定，减少因数据划分带来的波动
- 每个样本都参与训练和验证，数据利用率高
- 与模型无关，通用于所有监督学习算法

---

### ⚠️ 缺点

- 需要训练 K 次模型，计算开销比一次划分大
- 样本不均衡时可能出现验证集类别分布失衡
  - → 可以使用 StratifiedKFold 保证分布一致

---

### 🚀 常用于

- 模型选择、调参（GridSearchCV, RandomSearch）
- 模型验证阶段的准确性评估
- 样本量中等的泛化性能分析

---

### 💬 面试常见问题（含答案）

1. **什么是 K-Fold？**
   - 一种交叉验证方法，将数据划分为 K 份，每次取其中一份做验证，其余为训练，最后平均评估结果。

2. **K-Fold 比 Train/Test Split 好在哪？**
   - 更稳定，评估不依赖于一次划分的随机性；每个样本都被用来训练与验证。

3. **K 取多少比较合适？**
   - 通常取 5 或 10；样本少时可以更大，样本多时可以取 3~5 降低计算成本。

4. **K-Fold 会不会泄露数据？**
   - 若划分后才进行数据预处理（如归一化）即可避免泄露；错误顺序可能导致信息泄露。

5. **它和 LOOCV 有什么区别？**
   - LOOCV 是极限情况的 K-Fold（K=n），方差更高，计算代价也更大。



In [18]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import KFold

# 加载真实数据
X, y = fetch_california_housing(return_X_y=True)

# 初始化 KFold（10折）
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 显示每折划分的样本数量
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {fold + 1}:")
    print(f"  训练集大小: {len(train_index)}")
    print(f"  验证集大小: {len(test_index)}")
    print("-" * 30)


Fold 1:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 2:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 3:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 4:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 5:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 6:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 7:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 8:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 9:
  训练集大小: 18576
  验证集大小: 2064
------------------------------
Fold 10:
  训练集大小: 18576
  验证集大小: 2064
------------------------------


## 🧬 Leave-One-Out Cross Validation（LOOCV）

---

### 📘 介绍

LOOCV 是 K-Fold 的一个极端形式：

> 每次只留出一个样本作为验证集，其余所有样本都用于训练。共进行 $n$ 次训练验证（$n$ 是样本数），每次验证一个样本，最后对所有结果取平均。

---

### ⚙️ 原理

- 将整个数据集划分为 $n$ 个子集（每个子集只包含一个样本）
- 每次从中选择一个样本作为验证集，其余 $n-1$ 个样本训练模型
- 对所有 $n$ 次评估结果取平均，作为模型性能表现

---

### 🧠 特性

- 每个样本都被严格地用于验证一次
- 训练集最大化，每次只少一个样本
- 非常适合样本数量极少的任务

---

### ✅ 优点

- 几乎使用了所有数据做训练，数据利用率极高  
- 验证评估无偏性强（每个样本被独立评估）  
- 理论分析精确，适合统计建模

---

### ⚠️ 缺点

- **训练次数 = 样本数** → 计算成本极高  
- 每次验证只有 1 个样本 → 评估波动大（高方差）  
- 不适用于大样本数据或计算资源有限场景

---

### 🚀 常用于

- 医学 / 生物 / 金融等样本量极少领域
- 理论分析场景（如置信区间估计、无偏估计）
- 学术研究中的模型比较

---

### 💬 面试常见问题（含答案）

1. **LOOCV 是什么？**
   - Leave-One-Out 是每次用一个样本作为验证，其他样本用于训练的交叉验证方式。

2. **优缺点分别是什么？**
   - 优点：数据利用率极高、每个样本都被评估  
   - 缺点：训练开销大、评估结果波动大

3. **实际中你用过 LOOCV 吗？**
   - 在小样本任务或做 baseline 验证时会尝试，实际工业任务中更倾向使用 KFold。

4. **KFold 和 LOOCV 怎么选？**
   - 样本少 → LOOCV 更优；样本多或需效率 → KFold 更可取

5. **LOOCV 是否存在数据泄露风险？**
   - 只要在划分后进行数据预处理，就不会泄露（如标准化应在每轮内部处理）



In [19]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import LeaveOneOut

# 加载数据
X, y = fetch_california_housing(return_X_y=True)

# 初始化 LOOCV
loo = LeaveOneOut()

# 打印前 3 折的划分信息（数据量大，避免全部打印）
for i, (train_index, test_index) in enumerate(loo.split(X)):
    print(f"LOOCV 第 {i + 1} 折:")
    print(f"  训练集大小: {len(train_index)}")
    print(f"  验证集索引: {test_index}")
    print("-" * 30)

    if i == 2:  # 只展示前 3 折
        break


LOOCV 第 1 折:
  训练集大小: 20639
  验证集索引: [0]
------------------------------
LOOCV 第 2 折:
  训练集大小: 20639
  验证集索引: [1]
------------------------------
LOOCV 第 3 折:
  训练集大小: 20639
  验证集索引: [2]
------------------------------
